---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [11]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.loc[range(10,20)]

10    (5/11/85) Crt-1.96, BUN-26; AST/ALT-16/22; WBC...
11                        4/09/75 SOS-10 Total Score:\n
12    8/01/98 Communication with referring physician...
13    1/26/72 Communication with referring physician...
14    5/24/1990 CPT Code: 90792: With medical servic...
15    1/25/2011 CPT Code: 90792: With medical servic...
16          4/12/82 Total time of visit (in minutes):\n
17         1; 10/13/1976 Audit C Score, Highest/Date:\n
18                  4, 4/24/98 Relevant Drug History:\n
19    ) 59 yo unemployed w referred by Urgent Care f...
dtype: object

In [245]:
df[125]

's The patient is a 44 year old married Caucasian woman, unemployed Decorator, living with husband and caring for two young children, who is referred by Capitol Hill Hospital PCP, Dr. Heather Zubia, for urgent evaluation/treatment till first visit with Dr. Toney Winkler IN EIGHT WEEKS on 24 Jan 2001.\n'

In [246]:
import re
import numpy as np

def date_finder_mmddyy(line):
    return re.findall(r'\d{1,2}[/-]\d{1,2}[/-]\d{2}\b', line)

def date_finder_mmddyyyy(line):
    return re.findall(r'\d{1,2}[/-]\d{1,2}[/-]\d{4}\b', line)

def date_finder_mmyyyy(line):
    return re.findall(r'\d{1,2}[/-]\d{4}\b', line)

def date_finder_yyyy(line):
    return re.findall(r'\d{4}\b', line)

def date_finder_month(line):
    return re.findall(r'((?:\d{1,2} ))?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[?:, -]* )(\d{4})', line)

def date_finder_start_month(line):
    return re.findall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.]* )((?:\d{1,2}[?:, -]*)\d{4})', line)

def get_year(date):
    tokens = re.split('[,: \n\/-]+', date)
    if len(tokens) > 1:
        year = tokens[len(tokens)-1]
    else:
        year = date
    if len(year) < 4:
        year = '19' + year
    return year



def date_sorter():
    years = []
    for i in range(0,len(df)):
        x = date_finder_mmddyy(df[i])
        if len(x) == 0:
            x = date_finder_mmddyyyy(df[i])
        if len(x) == 0:
            x = date_finder_month(df[i])
        if len(x) == 0:
            x = date_finder_start_month(df[i])
        if len(x) == 0:
            x = date_finder_mmyyyy(df[i])
        if len(x) == 0:
            x = date_finder_yyyy(df[i])
        
        print(i, x[0])
        years.append(int(get_year(x[0])))
    
    years_series = pd.Series(years)
    years_sorted = years_series.sort_values()
    
    answer = pd.Series([i[0] for i in sorted(enumerate(years), key=lambda x:x[1])],np.arange(500))
    
    return answer

date_sorter()

0 03/25/93
1 6/18/85
2 7/8/71
3 9/27/75
4 2/6/96
5 7/06/79
6 5/18/78
7 10/24/89
8 3/7/86
9 4/10/71
10 5/11/85
11 4/09/75
12 8/01/98
13 1/26/72
14 5/24/1990
15 1/25/2011
16 4/12/82
17 10/13/1976
18 4/24/98
19 5/21/77
20 7/21/98
21 10/21/79
22 3/03/90
23 2/11/76
24 07/25/1984
25 4-13-82
26 9/22/89
27 9/02/76
28 9/12/71
29 10/24/86
30 03/31/1985
31 7/20/72
32 4/12/87
33 06/20/91
34 5/12/2012
35 3/15/83
36 2/14/73
37 5/24/88
38 7/27/1986
39 1-14-81
40 7-29-75
41 6/24/87
42 8/14/94
43 4/13/2002
44 8/16/82
45 2/15/1998
46 7/15/91
47 06/12/94
48 9/17/84
49 2/28/75
50 11/22/75
51 5/24/91
52 6/13/92
53 7/11/71
54 12/26/86
55 10/11/1987
56 3/14/95
57 12/01/73
58 12/5/2010
59 08/20/1982
60 7/24/95
61 8/06/83
62 02/22/92
63 6/28/87
64 07/29/1994
65 08/11/78
66 10/29/91
67 7/6/91
68 1/21/87
69 11/3/1985
70 7/04/82
71 4-13-89
72 7/11/77
73 4/12/74
74 09/19/81
75 9/6/79
76 12/5/87
77 01/05/1999
78 4/22/80
79 10/04/98
80 6/29/81
81 8/04/78
82 7/07/1974
83 09/14/2000
84 5/18/71
85 8/09/1981
86 6/05/93


TypeError: expected string or bytes-like object

In [241]:
def date_sorter2():
    
    a1_1 =df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{2})\b')
    a1_2 =df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{4})\b')
    a1 = pd.concat([a1_1,a1_2])
    a1.reset_index(inplace=True)
    a1_index = a1['level_0']
    
    a2 = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.]* )((?:\d{1,2}[?:, -]*)\d{4})')
    a2.reset_index(inplace=True)
    a2_index = a2['level_0']
    
    a3 = df.str.extractall(r'((?:\d{1,2} ))?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[?:, -]* )(\d{4})')
    a3.reset_index(inplace=True)
    a3_index = a3['level_0']
    
    a6 = df.str.extractall(r'(\d{1,2})[/](\d{4})')
    a6.reset_index(inplace=True)
    a6_index = a6['level_0']
    save=[]
    for i in a6_index:
        if not(i in a1_index.values):
            save.append(i)
    save = np.asarray(save)
    a6 = a6[a6['level_0'].isin(save)]

    
    a7_1= df.str.extractall(r'[a-z]?[^0-9](\d{4})[^0-9]')
    a7_2 = df.str.extractall(r'^(\d{4})[^0-9]')
    a7 = pd.concat([a7_1,a7_2])
    a7.reset_index(inplace=True)

    a7_index = a7['level_0']
    save=[]
    for i in a7_index:
        if not((i in a2_index.values) | (i in a3_index.values) | (i in a6_index.values)):
            save.append(i)
    save = np.asarray(save)
    a7 = a7[a7['level_0'].isin(save)]
    
    s = a1.level_0.values.tolist()+a2.level_0.values.tolist()+a3.level_0.values.tolist()+a6.level_0.values.tolist()+a7.level_0.values.tolist()
    s = np.asarray(s)
    
    a1.columns=['level_0','match','month','day','year']
    a1['year']=a1['year'].apply(str)
    a1['year']=a1['year'].apply(lambda x: '19'+x if len(x)<=2 else x)
   
    a2[1] = a2[1].apply(lambda x: x.replace(',',''))
    a2['day'] = a2[1].apply(lambda x:x.split(' ')[0])
    a2['year'] = a2[1].apply(lambda x:x.split(' ')[1])
    a2.columns=['level_0','match','month','day-year','day','year']
    a2.drop('day-year',axis=1,inplace=True) 
    
    a3.columns=['level_0','match','day','month','year']
    a3['day'] = a3['day'].replace(np.nan,-99)
    a3['day'] = a3['day'].apply(lambda x: 1 if int(x)==-99 else x)

    a3['month'] = a3.month.apply(lambda x: x[:3])
    a3['month'] = pd.to_datetime(a3.month, format='%b').dt.month
    
    a6.columns=['level_0','match','month','year']
    a6['day']=1
  
    a7.columns=['level_0','match','year']
    a7['day']=1
    a7['month']=1
   
    final = pd.concat([a1,a2,a3,a6,a7])
    final['date'] =pd.to_datetime(final['month'].apply(str)+'/'+final['day'].apply(str)+'/'+final['year'].apply(str))
    final = final.sort_values(by='level_0').set_index('level_0')

    myList = final['date']
    answer = pd.Series([i[0] for i in sorted(enumerate(myList), key=lambda x:x[1])],np.arange(500))
    return answer

date_sorter2()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    255
486    439
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64